In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import funcs
import imp
import random
import mne
from scipy.signal import hilbert
imp.reload(funcs)
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm


# Import the FOOOF object
from fooof import FOOOF

# Import some internal functions
#   These are used here to demonstrate the algorithm
#   You do not need to import these functions for standard usage of the module
from fooof.sim.gen import gen_aperiodic
from fooof.plts.spectra import plot_spectra
from fooof.plts.annotate import plot_annotated_peak_search

C:\Users\Tianyi Zheng\AppData\Local\Temp\ipykernel_19348\1452165549.py:11: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [ ]:
load_path = os.path.join('..', '..', '..', 'data', 'subjects_rest_eeg.pkl')
subjects_rest_eeg = pd.read_pickle(load_path)
# 7.4s

In [4]:
exp_info = funcs.load_info()
exp_info["exponent pre"] = None
exp_info["exponent post"] = None

exp_info["R_squared pre"] = None
exp_info["R_squared post"] = None

exp_info["error pre"] = None
exp_info["error post"] = None

In [ ]:
task_time_name = ['pre', 'post']
for subject_id in range(1, 27):
    for tasktime in range (1, 3):
        raw = subjects_rest_eeg[subjects_rest_eeg['subject id'] == subject_id][task_time_name[tasktime-1]].values[0]
        print("\nsubject_id: ", subject_id, " tasktime: ", task_time_name[tasktime-1], "\n")

        eeg = raw.get_data()[1:32,:]
        exponent = np.zeros((eeg.shape[0]))
        R_squared = np.zeros((eeg.shape[0]))
        error = np.zeros((eeg.shape[0]))

        # Compute the power spectrum with multitaper method
        psds, freqs = mne.time_frequency.psd_array_multitaper(eeg, sfreq=4800, fmin=1, fmax=100, n_jobs=1)
        # Define frequency range across which to model the spectrum
        freq_range = [3, 40]
        for channel in range(eeg.shape[0]):
            # Initialize FOOOF object
            fm = FOOOF()
            # Fit FOOOF model
            fit_results = fm.fit(freqs, psds[channel,:], freq_range)
            exponent[channel] = fm.get_results().aperiodic_params[1]
            R_squared[channel] = fm.get_results().r_squared
            error[channel] = fm.get_results().error

        idx = exp_info[(exp_info['subject id'] == subject_id)].index[0]
        exp_info.at[idx, 'exponent '+task_time_name[tasktime-1]] = exponent
        exp_info.at[idx, 'R_squared '+task_time_name[tasktime-1]] = R_squared
        exp_info.at[idx, 'error '+task_time_name[tasktime-1]] = error


In [ ]:
save_path = os.path.join('..', '..', '..', 'data', 'subjects_rest_fooof' +'.pkl')
exp_info.to_pickle(save_path)